In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import requests
import os
import numpy as np

import torch
from torchvision import models
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax
from torch.optim import Adam,lr_scheduler
from torch.nn import functional as F
from torchvision import transforms
from torch.utils.data import Dataset

In [21]:
Deeplabv3plus_path = r'E:\codes\python\area51m\seg_halfprecision\plant-segmentation'
Data_path = r'E:\codes\python\area51m\seg_halfprecision\plant-segmentation'
TrainingLabelNames = 'TrainingLabelNames.csv'
TestLabelNames = 'TestLabelNames.csv'
Training_input_folder = 'input'
Training_label_folder = 'label'
Test_input_folder = 'input_test'
Test_label_folder = 'label_test'
ModelPath = r'E:\data\MODELS' 
OneStageModelName = 'onestage_model'

In [8]:
from utils import *
import sys
import os

device = torch.device("cuda:0")

sys.path.append(Deeplabv3plus_path)
sys.path.append(Deeplabv3plus_path+r'\pytorch_deeplab_xception')
from pytorch_deeplab_xception.modeling import deeplab

os.environ['TORCH_HOME'] = ModelPath

In [9]:
num_c = 8

dl = deeplab.DeepLab(num_classes=num_c,backbone = 'resnet',output_stride = 16).to(device)

for name, param in dl.named_parameters():                
    if name.startswith('backbone'):
        param.requires_grad = False

In [10]:
image_size = (900,900)

transform = transforms.Compose([transforms.Resize(image_size),
                                transforms.ColorJitter(brightness = 0.15,
                                                       saturation = 0.1,
                                                       hue = 0.01,
                                                       contrast = 0.15),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [22]:
data = pd.read_csv(r'{0}\{1}'.format(Data_path,TrainingLabelNames))

dataset_train = Segdata(transform,image_size,
                        TrainingLabelNames,Data_path,[Training_input_folder,Training_label_folder])

dataset_test = Segdata(transform,image_size,
                       TestLabelNames,Data_path,[Test_input_folder,Test_label_folder])

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=6, 
                                         shuffle=True, num_workers=0,drop_last  = True)

dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, num_workers=0,drop_last  = False)

In [13]:
freq = 1/get_frequency(data,image_size,num_c,Data_path,Training_label_folder)

In [16]:
plist = [
         {'params': dl.parameters(),'lr':1e-2}
         ]

criterion = CrossEntropyLoss(ignore_index = 0,weight = torch.log(freq)/torch.log(freq).sum())

optimizer = Adam(params=plist)
        
scheduler = lr_scheduler.StepLR(optimizer, step_size=50,gamma=0.1)

loss_list_train = []

loss_list_test = []

best_val_loss = float('inf')

n_epoch = 100

In [23]:
for epoch in range(n_epoch):
    training_loss,training_iou = train(dl,dataloader_train,optimizer,criterion,loss_list_train,num_c)
    val_loss,val_iou = evaluate(dl,dataloader_test,criterion,loss_list_test,num_c)
    scheduler.step()
    print('finished {0}th training'.format(epoch+1))
    print('training loss: {0}, training iou: {1}'.format(training_loss.item(),training_iou))
    print('validation loss: {0}, validation iou: {1}'.format(val_loss.item(),val_iou))
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(dl.state_dict(), r'{0}\{1}_best.pt'.format(ModelPath,OneStageModelName))
torch.save(dl.state_dict(), r'{0}\{1}_last.pt'.format(ModelPath,OneStageModelName))

finished 1th training
training loss: 0.3158927341302236, training iou: 0.44726441992047544
validation loss: 0.27931625880301, validation iou: 0.48960903927472144
